<a href="https://colab.research.google.com/github/appersaravanan/Semeter-2-Assignment-1/blob/master/custom_object_dectection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
# !wget https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
# !wget http://imagenet.stanford.edu/internal/car196/cars_train.tgz
# (train_ds, val_ds, test_ds), metadata = tfds.load(
#     'cars196',
#     split=['train[:80%]'],
#     with_info=True
# )
from google.colab import drive
drive.mount('/content/drive')
# https://github.com/deanwetherby/tf_oda_stanford_cars

# https://towardsdatascience.com/how-to-train-your-own-object-detector-with-tensorflows-object-detector-api-bec72ecfe1d9
# https://github.com/Tony607/object_detection_demo


Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from keras.applications import VGG16
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import  MaxPooling2D,UpSampling2D,Input,BatchNormalization,Conv2D,Activation,MaxPool2D,Flatten,Dense,GlobalAveragePooling2D,Lambda,Concatenate,GlobalMaxPooling2D,Add,multiply
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.preprocessing import image as image_utils
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import VGG16
import cv2,os,tempfile
import requests
from keras.preprocessing import image
from scipy.io import loadmat
import tensorflow as tf
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import  Reshape
import hashlib
import io
import logging
import csv
import PIL.Image


In [ ]:
img_width, img_height = 224, 224
nb_train_samples = 8144
nb_validation_samples = 4899
epochs = 10
batch_size = 64
n_classes = 196
path_base='/content/drive/MyDrive/semester_2_final_project/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cp /content/drive/MyDrive/YOLOv4_weight/data/cars_train.tgz /content/drive/MyDrive/semester_2_final_project/data/training_set/
%cd /content/drive/MyDrive/semester_2_final_project/data/training_set/
!tar xvzf cars_train.tgz



In [ ]:
%cp /content/drive/MyDrive/semester_2_final_project/cars_test.zip /content/drive/MyDrive/semester_2_final_project/data/test_set/
%cd /content/drive/MyDrive/semester_2_final_project/data/test_set/
!sudo unzip cars_test.zip


In [ ]:
%cd /content/drive/MyDrive/semester_2_final_project/


/content/drive/MyDrive/semester_2_final_project


In [ ]:


!rm -f car_devkit.tgz
!wget https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
!tar xvzf car_devkit.tgz

--2021-04-29 16:40:50--  https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330960 (323K) [application/x-gzip]
Saving to: ‘car_devkit.tgz’

car_devkit.tgz      100%[===================>] 323.20K  1.53MB/s    in 0.2s    

2021-04-29 16:40:51 (1.53 MB/s) - ‘car_devkit.tgz’ saved [330960/330960]

devkit/
devkit/cars_meta.mat
devkit/cars_train_annos.mat
devkit/cars_test_annos.mat
devkit/README.txt
devkit/train_perfect_preds.txt
devkit/eval_train.m


In [ ]:
# !wget http://imagenet.stanford.edu/internal/car196/cars_train.tgz

# !tar xvzf cars_train.tgz
# !rm -f cars_train.tgz
# !wget http://imagenet.stanford.edu/internal/car196/cars_test.tgz
# !tar xvzf cars_test.tgz
# !rm -f cars_test.tgz

In [ ]:
%cd {path_base}'data/inputs'
assert os.getcwd()=='/content/drive/MyDrive/semester_2_final_project/data/inputs', 'Directory should be "/content/drive/MyDrive/semester_2_final_project/data/inputs" instead of "{}"'.format(os.getcwd())
!rm -Rfv ./*
!pwd
annotsa = loadmat('/content/drive/MyDrive/semester_2_final_project/devkit/cars_meta.mat')
i=1
classes_name=[]
labelMap='';
for name in annotsa['class_names'][0]:
  dir_name=str(i)+'_'+str(name[0]).replace(' ','_')
  os.makedirs(dir_name)
  if i<10:
    classes_name.append(dir_name)  
    labelMap+=' item { \n \t id:'+ str(i) +' \n \t name:' +" '" + dir_name +"' \n } \n \n"
  i=i+1
labelMap



In [ ]:
f= open("/content/drive/MyDrive/semester_2_final_project/data/annotations/train_label_map_10.pbtxt","w+")
f.write(labelMap)
f.close()

In [ ]:
import shutil
from shutil import copyfile

from_train_data_dir = '/content/drive/MyDrive/semester_2_final_project/data/training_set/cars_train'
to_directory ='/content/drive/MyDrive/semester_2_final_project/data/inputs'

annos = loadmat('/content/drive/MyDrive/semester_2_final_project/devkit/cars_train_annos.mat')
data = [[row.flat[0] for row in line] for line in annos['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class', 'fname']
df_train = pd.DataFrame(data, columns=columns)
gp=df_train.groupby('class')
files=""
# for index,classes in gp:
    
#     files=classes['fname']
#     i=0
#     for name,file in files.items():
#         print(file)
#         copyfile('{}/{}'.format(from_train_data_dir,file),'{}/{}/{}'.format(to_directory,classes_name[index-1],file))
        
#         i=i+1


In [ ]:
trimmed_data_set= df_train.loc[df_train['class']<10]

In [ ]:
trimmed_data_set

,fname,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class
1,00002.jpg,36,116,868,587,3
64,00065.jpg,63,175,417,320,8
68,00069.jpg,142,124,675,445,5
80,00081.jpg,10,55,540,271,8
149,00150.jpg,11,208,596,386,8
...,...,...,...,...,...,...
8017,08018.jpg,181,51,427,232,6
8030,08031.jpg,12,16,117,71,1
8095,08096.jpg,25,118,319,232,8
8126,08127.jpg,38,29,380,177,5


In [ ]:
print(trimmed_data_set.columns.to_list())

columns = ['fname','bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class']
trimmed_data_set=trimmed_data_set[columns]
print(trimmed_data_set.columns.to_list())
print(trimmed_data_set.to_csv())
f= open("/content/drive/MyDrive/semester_2_final_project/data/annotations/train.csv","w+")
f.write(trimmed_data_set.to_csv())
f.close()



In [ ]:
%cd /content/drive/MyDrive/semester_2_final_project/tf_model

!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install tf_slim

%cd /content/drive/MyDrive/semester_2_final_project/tf_model/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/drive/MyDrive/semester_2_final_project/tf_model/models:/content/drive/MyDrive/semester_2_final_project/tf_model/models/research/:/content/drive/MyDrive/semester_2_final_project/tf_model/models/research/slim/'

import sys
sys.path.append("/content/drive/MyDrive/semester_2_final_project/tf_model/models")

!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
%cd /content/drive/MyDrive/semester_2_final_project/tf_model/models/research
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

In [ ]:


from object_detection.utils import dataset_util
from object_detection.utils import label_map_util


SETS = ['train', 'test', 'merged']
def dict_to_tf_example(annotation, dataset_directory, label_map_dict):
  im_path = str(annotation['fname'])
  cls = int(annotation['class'])
  x1 = int(annotation['bbox_x1'])
  y1 = int(annotation['bbox_y1'])
  x2 = int(annotation['bbox_x2'])
  y2 = int(annotation['bbox_y2'])

  # read image
  full_img_path = os.path.join(dataset_directory, im_path)

  # read in the image and make a thumbnail of it
  max_size = 300, 300
  big_image = PIL.Image.open(full_img_path)
  width,height = big_image.size
  big_image.thumbnail(max_size, PIL.Image.ANTIALIAS)
  full_thumbnail_path = os.path.splitext(full_img_path)[0] + '_thumbnail.jpg'
  big_image.save(full_thumbnail_path)

  with tf.io.gfile.GFile(full_thumbnail_path, 'rb') as fid:
    encoded_jpg = fid.read()
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = PIL.Image.open(encoded_jpg_io)

  xmin = []
  xmax = []
  ymin = []
  ymax = []

  # calculate box using original image coordinates
  xmin.append(max(0, x1 / width))
  xmax.append(min(1.0, x2 / width))
  ymin.append(max(0, y1 / height))
  ymax.append(min(1.0, y2 / height))

  # set width and height to thumbnail size for tfrecord ingest
  width,height = image.size

  classes = []
  classes_text = []

  label=''
  for name, val in label_map_dict.items():
    if val == cls: 
      label = name
      break

  classes_text.append(label.encode('utf8'))
  classes.append(label_map_dict[label])
  
  example = tf.train.Example(features=tf.train.Features(feature={
	'image/height': dataset_util.int64_feature(height),
	'image/width': dataset_util.int64_feature(width),
	'image/filename': dataset_util.bytes_feature(full_img_path.encode('utf8')),
	'image/source_id': dataset_util.bytes_feature(full_img_path.encode('utf8')),
	'image/encoded': dataset_util.bytes_feature(encoded_jpg),
	'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
	'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
	'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
	'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
	'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
	'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
	'image/object/class/label': dataset_util.int64_list_feature(classes),
  }))
  return example 

data_dir = '/content/drive/MyDrive/semester_2_final_project/data/training_set/cars_train'
csv_file = '/content/drive/MyDrive/semester_2_final_project/data/annotations/train.csv'

writer =  tf.compat.v1.python_io.TFRecordWriter('/content/drive/MyDrive/semester_2_final_project/data/annotations/stanford_cars.tfrecord')
label_map_dict = label_map_util.get_label_map_dict('/content/drive/MyDrive/semester_2_final_project/data/annotations/train_label_map_10.pbtxt')
count=0;
with open(csv_file) as f:
  csv_reader = csv.DictReader(f)
  for row in csv_reader:
    tf_example = dict_to_tf_example(row, data_dir, label_map_dict)
    writer.write(tf_example.SerializeToString())
    count+=1;
    print(count)
writer.close()

In [ ]:
%cd /content/drive/MyDrive/semester_2_final_project/data/annotations
%cp stanford_cars.tfrecord  stanford_cars_test.tfrecord


/content/drive/MyDrive/semester_2_final_project/data/annotations


In [ ]:
import tensorflow as tf 
raw_dataset = tf.data.TFRecordDataset("/content/drive/MyDrive/semester_2_final_project/data/annotations/stanford_cars.tfrecord")

count=0;
for raw_record in raw_dataset:
    # example = tf.train.Example()
    # example.ParseFromString(raw_record.numpy())
    count+=1;
    print(count)

In [ ]:
%cd /content/drive/MyDrive/semester_2_final_project/temp

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL ='ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'


DEST_DIR = '/content/drive/MyDrive/semester_2_final_project/temp'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

!tar xvzf {MODEL_FILE}

In [ ]:
!pip install lvis
!pip install tensorflow-addons

In [ ]:

# --checkpoint_dir=/content/drive/MyDrive/semester_2_final_project/temp/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/checkpoint \
!python /content/drive/MyDrive/semester_2_final_project/tf_model/models/research/object_detection/model_main_tf2.py \
--pipeline_config_path=/content/drive/MyDrive/semester_2_final_project/temp/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config \
--model_dir=/content/drive/MyDrive/semester_2_final_project/temp/fine_tune_model  --num_train_steps=2000


2021-04-30 17:45:26.815477: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-30 17:45:30.227463: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-30 17:45:30.228581: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-30 17:45:30.238060: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-30 17:45:30.238099: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (e53ff65ff2fd): /proc/driver/nvidia/version does not exist
2021-04-30 17:45:30.240476: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
W0430 17:45:30.242950 139829078566784 cross_device_ops.py:1321] There are no

In [ ]:
PIPELINE_CONFIG_PATH='/content/drive/MyDrive/semester_2_final_project/temp/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config'
MODEL_DIR='/content/drive/MyDrive/semester_2_final_project/temp/fine_tune_model'
!ls {MODEL_DIR}

checkpoint		    ckpt-2.data-00000-of-00001	ckpt-3.index
ckpt-1.data-00000-of-00001  ckpt-2.index		train
ckpt-1.index		    ckpt-3.data-00000-of-00001


In [ ]:
!python /content/drive/MyDrive/semester_2_final_project/tf_model/models/research/object_detection/exporter_main_v2.py \
 --input_type encoded_image_string_tensor \
 --pipeline_config_path {PIPELINE_CONFIG_PATH} \
 --trained_checkpoint_dir  '/content/drive/MyDrive/semester_2_final_project/temp/fine_tune_model' \
 --output_directory /content/drive/MyDrive/semester_2_final_project/model_export

In [1]:
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

ModuleNotFoundError: ignored

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def get_keypoint_tuples(eval_config):
  """Return a tuple list of keypoint edges from the eval config.
  
  Args:
    eval_config: an eval config containing the keypoint edges
  
  Returns:
    a list of edge tuples, each in the format (start, end)
  """
  tuple_list = []
  kp_list = eval_config.keypoint_edge
  for edge in kp_list:
    tuple_list.append((edge.start, edge.end))
  return tuple_list

In [ ]:
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)